### 1. 색상 공간 변환

``` python
dst = cv2.cvtColor(
    src,
    code,
    dstCn = None
)
```

In [2]:
import cv2

src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\crow.jpg")
dst = cv2.cvtColor(src, cv2.COLOR_BGR2HLS)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 2. HSV 색상 공간

- 색상(Hue)  
일반적으로 0~360 사이의 값으로 표현  
but, 8비트 이미지로눈 0~255의 값만 할당 가능  
=> 255의 절반인 0~179 범위로 사용.

    ※ **색상의 최솟값과 최댓값에서 빨간색 범위가 겹침. (원형 형태)**

- 채도(Saturation): 0~255 범위
- 명도(Value): 0~255 범위

  
``` python
# 채널 분리 함수
mv = cv2.split(
    src
)

# 채널 병합 함수
dst = cv2.merge(
    mv
)

# 리스트 반환 형식 변경
c0, c1, c2 = cv2.split(src)

# 리스트 형식 입력
dst = cv2.merge([c0, c1, c2])

# 배열 요소의 범위 설정
dst = cv2.inRange(
    src,
    lowerb, # 낮은 범위
    upperb  # 높은 범위
)
```




In [3]:
# Hue 공간 색상 검출

src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\tomato.jpg")
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)


h, s, v = cv2.split(hsv)
h_red = cv2.inRange(h, 0, 5)

dst = cv2.bitwise_and(hsv, hsv, mask=h_red)
dst = cv2.cvtColor(dst, cv2.COLOR_HSV2BGR)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

``` python
# 배열 병합 함수
dst = cv2.addWeighted(
    src1,
    alpha,
    src2,
    beta,
    gamma,
    dtype = None
)
```
배열 병합 함수 수식
dst = src1 x alpha + src2 x beta + gamma

In [4]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\tomato.jpg")
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

h, s, v = cv2.split(hsv)

orange = cv2.inRange(hsv, (8, 100, 100), (20, 255, 255))
blue = cv2.inRange(hsv, (110, 100, 100), (130, 255, 255))
mix_color = cv2.addWeighted(orange, 1.0, blue, 1.0, 0.0)

dst = cv2.bitwise_and(hsv, hsv, mask = mix_color)
dst = cv2.cvtColor(dst, cv2.COLOR_HSV2BGR)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 3. 이진화

임계값보다 낮은 픽셀값은 0이나 원본 픽셀값으로 변경, 임계값 보다 높은 픽셀값은 최댓값으로 변경
``` python
retval, dst = cv2.threshold(
    src,
    thresh, # 임계값
    maxval, # 최댓값
    type
)
```

일반적으로 **단일 채널 이미지**에서 활용  
**다중 채널 이미지**에서 이진화를 적용할 경우 각 채널을 분히해서 이진화 함수를 적용한 후 이미지를 다시 병합해서 반환

In [5]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\swan.jpg")
_, binary = cv2.threshold(src, 127, 255, cv2.THRESH_BINARY)

cv2.imshow("binary", binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

알고리즘은 디지털 영상 처리에서 자세히 공부하기...
- 오츠 알고리즘
    - 이미지의 밝기 분포(히스토그램)을 통해 최적의 임곗값을 찾아 이진화하는 알고리즘
    - 가능한 모든 임계값을 고려해서 이미지 내의 픽셀들을 두 개의 클래스로 분류했을 때 클래스 간의 분산을 최소화하거나 차이를 최대화하는 임곗값
    - 가능한 모든 임계값을 고려하므로 느림.
    - 수식: **σ² = α x σ₁² + β x σ₂²** (이건 클래스 내의 분산 아닌가)
    - Bimoadl Image에서 사용하기 적합 (Bimoadl Image" 히스토그램상에서 두개의 피크를 치는 형태의 이미지)

정확히는 모르겠음.
- 삼각형 알고리즘
    - 이미지의 밝기 분포(히스토그램)을 통해 최적의 임곗값을 찾아 이진화하는 알고리즘.
    - 히스토그램에서 최대 거리를 구성할 수 있는 임계값을 찾아 이진화를 적용.
    - 삼각형의 빗변 사이의 거리가 최대일때 수직인 선이 히스토그램의 최대 거리
    - 즉, 히스토그램에 그려진 선 사이의 거리가 최대인 지역값이 임계값


- 적응형 이진화 알고리즘
    - 어떠한 임계값을 주더라도 이진화 처리가 어려운 이미지에 적용
    - 조명의 변화나 반사가 심한 경우 이미지내 밝기 분보가 달라 국소적으로 임계값을 적용해야함.
    - 각 픽셀 주변의 blockSize x blockSize영역에 대한 가중 평균 계산, 상수C를 감산해서 적응형 임계값 T(x, y)설정
    - blockSize가 너무 크면 연산 시간이 오래걸림.
    - 상수 C는 일반적으로 양수값, 경우에 따라 0이나 음수도 가능
    -  상수 C가 음수이면 전체 영역 어두어짐. 

``` python
cv2.adaptiveThreshold(
    Mat src,
    Mat dst,
    double maxValue,
    AdaptiveThresholdTypes adaptiveMethod,
    ThresholdTypes thresholdType,
    int bolckSize,  #   blockSize x blockSize 영역
    double C    #    상수 C 
)

```

In [9]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\swan.jpg")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 33, -5)

cv2.imshow("binary", binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 4. 이미지 연산

Numpy클래스의 배열 연산과 동일하거나 비슷한 의미와 결과

- 덧셈 함수  
    ``` python
    Cv2.Add(Mat src1, Mat src2, Mat dst, Mat mask = null, int dtype = -1)
    ```
    배열과 배열 or 배열과 스칼라의 요소별 합 계산.  
    dtype형식의 배열 dst에 저장.  
    정밀도에 따라 요소의 최댓값과 최소값이 존재하면 이 값들을 넘어갈 수 없다.  
    mask 배열의 요소값이 0이 아닌 경우에만 연산이 유효.

- 뺄셈 함수
    ``` python
    Cv2.Subtract(Mat src1, Mat src2, Mat dst, Mat mask = null, int dtype = -1)
    ```

- 곱셈 함수  
    ``` python
    dst = cv2.multiply(src1, src2, scale = None, dtype = None)
    ```
    배열과 배열 또는 배열과 스칼라의 요소별 곱에 추가로 scale을 곱한 값을 계산  
  
- 나눗셈 함수
    ``` python
    dst = cv2.divide(src1, src2, scale = None, dtype = None)

- 최댓값 함수
    ``` python
    dst = cv2.max(src1, src2)
    ```

- 최솟값 함수
    ``` python
    dst = cv2.min(src1, src2)
    ```
  
- 최소/최대 위치 반환 함수  
    ``` python
    minVal, maxVale, minLoc, macLoc = np.minMaxLoc(src)
    ```
    배열의 초솟값과 최대값의 위치와 값을 반환  

- 절댓값 함수
    ``` python
    dst = np.abs(src)
    ```
    
- 절댓값 차이 함수
    ``` python
    dst = cv2.absdiff(src1, src2)
    ```

- 비교 함수
    ``` python
    dst = cv2.compare(src1,src2, cmpop)
    ```
    cmpop  비교함 수 플래그를 이용해 모든 요소에 대해 일대일 배교연산ㅇ르 수행할 수 있음.  
    비교 결과 False이면 요소값을 0으로 변경, True이면 255로 변경.  
    
- 선형 방정식 시스템의 해 찾기 함수
    ``` python
    successs, dst = cv2.Solve(src1, src2, flags=None)
    ```
    역함수를 기반으로 선형 시스템의 해를 빠르게 구해서 반환  
    부동 소수점 형식 (float, double)만 지원  

- AND 연산 함수
    ``` python
    dst = cv2.bitwise_and(src1, src2, mask=None)
    ```
- OR 연산 함수
    ``` python
    dst = cv2.bitwise_or(src1, src2, mask=None)
    ```
- XOR 연산 함수
    ``` python
    dst = cv2.bitwise_xor(src1, src2, mask=None)
    ```
- NOT 연산 함수
    ``` python
    dst = cv2.bitwise_not(src, mask=None)
    ```

In [15]:
import numpy as np

src1 = np.array([[9,2], [1,1]], dtype=np.double)
src2 = np.array([38,5], dtype=np.double)

dst = cv2.solve(src1, src2, flags=cv2.DECOMP_LU)    # 가우스 소거
print(dst)

(True, array([[4.],
       [1.]]))


### 5. 흐림 효과
- 커널과 고정점
- 테두리 외삽법

- 종류
    1. 단순 흐림 효과  
    입력 이미지의 각 픽셀에 대한 커널을 적용해 모든 픽셀의 **단순 평균**을 구해 출력이미지에 저장
        ``` python
        dst = cv2.blur(
            src,
            ksize,  # 커널 크기
            anchor = None,  # 고정점 설정, None이면 커널을 기준으로 중앙에 위치
            borderType = None   # 테두리 외삽법
        )
        ```
    
    2. 박스 필터 흐림 효과  
    커널의 내부 값이 모두 같은 필터
        ``` python
        dst = cv2.boxFilter(
            src,
            ddepth, # 출력이미지의 정밀도(cv2.CV_8U 형식, -1로 설정하면 입력 이미지와 동일한 정밀도)
            ksize,
            anchor = None,
            normalize = None,   # 정규화된 박스 필터 -> 커널의 모드 값이 커널의 개수만큼 나눠짐.
            borderType = Noe
        )
        ```
    
    3. 중간값 흐림 효과  
    중심 픽셀 주변으로 사각형 크기의 이웃한 픽셀들의 중간값을 사용해 각 픽셀의 값을 변경  
    즉, 고정점이 항상 커널의 중심에 있다고 가정.  
        ``` python
        dst = cv2.medianBlur(
            src,    # 2차원 이하 배열만 가능
            ksize   # ksize는 홀수 값만 가능(커널에서 중간에 있는 값을 선택하므로)
        )
        ```    

    4. 가우시안 흐림 효과
    흐림 효과 함수에서 가장 유요한 함수  
    이미지의 각 지점에 가우시안 커널을 적용해 합산후 반환  
        ``` python
        dst = cv2.GaussianBlur(
            src,
            ksize,          # 0보다 커야하고 홀수만 가능
            sigmaX,         # x방향의 가우스 커널 표준 편차
            sigmaY = None,  # y방향의 가우스 커널 표준 변차 # 0인경우 sigmaX와 같아짐.
                            # sigmaX, sigmaY 모두 0이면 커널의 크기에 의해 자동으로 설정됨.
            borderType = Noe

        )

        ```
    5. 양방향 필터 흐림 효과
    **가장자리를 선명하게 보존**하면서 노이즈를 우수하게 제거하는 흐림 효과 함수.  
    두 종류의 가우시안 필터로 흐림 효과를 적용  
    가우시안 가중치는 픽셀의 위치와 해당 위치의 값을 사용  
    ** 느림, 지름(d)이 클수록 수채화처럼 변형
        ``` python
        dst = cv2.bilateralFilter(
            src,
            d,  // 흐림 효과를 적용할 각 필셀 영역의 지름   # 동영상 5 # 실시간 처리x 9   # -1로 지정할 경우 시그마 공간과 비례하도록 설정됨.
            sigmaColor, # 색상 공간에서 사용할 가우시안 커널의 너비    # 클수록 흐림 효과에 포함될 강도의 범위가 넓어짐
            sigmaSpace, # 좌표 공간에서 사용할 가우시안 커널의 너비    # 클수록 인접한 픽셀에 영향을 미침
            dst = None,
            borderType = None
        )
        ```

In [16]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\crescent.jpg")

# 양방향 필터 흐림 효과 함수
dst = cv2.bilateralFilter(src, 100, 33, 11, borderType=cv2.BORDER_ISOLATED)
# 픽셀의 지름(d): 100
# 시그마 색상 33
# 시그마 공간 11

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()